# **1. Consensus atom mapping**

In [ ]:
import sys

sys.path.append("../../")
from syntemp.pipeline import run_aam
from syntemp.SynUtils.utils import load_database

data = load_database(f"../../Data/Testcase/demo.json.gz")[:10]
mapped_reactions = run_aam(
    data,
    mapper_types=["local_mapper", "rxn_mapper", "graphormer"],
    rsmi_column="reactions",
)

# **2. ITSG and Rule Extraction**

Assume that you run consensus atom mapping (take time)

## 2.1. Extraction

In [ ]:
import sys

sys.path.append("../../")
from syntemp.SynAAM.aam_postprocess import AMMPostprocessor
from syntemp.SynITS.its_extraction import ITSExtraction
from syntemp.SynUtils.utils import load_database, save_database

data = mapped_reactions
mapper_name = ["rxn_mapper", "graphormer", "local_mapper"]

# make sure reactions all have atom mapping
check_valid = AMMPostprocessor.parallel_postprocess(
    data, mapper_name, threshold=3, n_jobs=5, verbose=1
)
valid_data = [reaction for reaction in check_valid if reaction.get("Valid")]

In [ ]:
# Extract rules and graph rules based on Imaginary Transition State (ITS)
its_graph, its_graph_wrong = ITSExtraction.parallel_process_smiles(
    data, mapper_name, n_jobs=5, verbose=1, export_full=False, check_method="RC"
)

len(its_graph)

In [ ]:
from syntemp.SynVis.chemical_graph_visualizer import ChemicalGraphVisualizer

vis_graph = ChemicalGraphVisualizer(seed=42)
fig = vis_graph.visualize_all(its_graph[0]["ITSGraph"], its_graph[0]["GraphRules"])
fig

In [ ]:
vis_graph = ChemicalGraphVisualizer(seed=42)
fig = vis_graph.vis_three_graph(its_graph_wrong[0]["rxn_mapper"])
display(fig)

In [ ]:
vis_graph.vis_three_graph(its_graph_wrong[0]["local_mapper"])

## 2.2. Rules Extension

In [ ]:
from syntemp.SynRule.rules_extraction import RuleExtraction

its_graph_rules = RuleExtraction.process_rules_extraction(
    its_graph, mapper_type="ITSGraph", n_jobs=1, verbose=0, extend=True, n_knn=1
)
vis_graph.vis_three_graph(
    its_graph_rules[0]["GraphRules"],
    left_graph_title="L",
    right_graph_title="R",
    k_graph_title="K",
)

# **3. ITS Hydrogen Adjuster**

Make sure ITSG be a cyclic graph

## 3.1. Cyclic ITS Graph

In [ ]:
from syntemp.SynITS.its_hadjuster import ITSHAdjuster

from copy import deepcopy

test_list = deepcopy(its_graph)
print(len(test_list))

process_graph_data = ITSHAdjuster.process_graph_data_parallel(
    test_list, "ITSGraph", n_jobs=5, verbose=2
)
process_graph_data = [
    value for _, value in enumerate(process_graph_data) if value["ITSGraph"] is not None
]
len(process_graph_data)

In [ ]:
vis_graph.visualize_all(
    process_graph_data[2]["ITSGraph"], process_graph_data[2]["GraphRules"]
)

## 2.2. Uncertain atom mapping refinement

In [ ]:
from syntemp.SynITS.its_refinement import ITSRefinement
from syntemp.SynUtils.utils import load_from_pickle

In [ ]:
from syntemp.SynVis.chemical_graph_visualizer import ChemicalGraphVisualizer

vis = ChemicalGraphVisualizer(seed=42)
vis.vis_three_graph(its_graph_wrong[0]["local_mapper"])

In [ ]:
vis.vis_three_graph(its_graph_wrong[0]["rxn_mapper"])

In [ ]:
process_graphs = ITSRefinement.process_graphs_in_parallel(
    its_graph_wrong, n_jobs=1, verbose=1
)
print(len(process_graphs))

process_graphs = [
    value for key, value in enumerate(process_graphs) if value is not None
]
print(len(process_graphs))

In [ ]:
vis.vis_three_graph(process_graphs[0]["GraphRules"])

# **4. Graph Modelling Language - MØD_rules** 

In [ ]:
from syntemp.SynRule.rule_writing import RuleWriting

results = RuleWriting.auto_extraction(
    process_graph_data,
    reindex=True,
    save_path=None,
    rule_column="GraphRules",
    n_jobs=1,
    attributes=["charge", "isomer"],
)
print(results[0])

# **5. MolToGraph**

In [ ]:
from syntemp.SynUtils.utils import load_database, load_from_pickle
from syntemp.SynChemistry.mol_to_graph import MolToGraph
from syntemp.SynChemistry.graph_to_mol import GraphToMol

graph_test = its_graph_rules[0]["GraphRules"][2]

In [ ]:
from rdkit import Chem

converter = MolToGraph()
smiles = "[NH2:4][c:5]1[cH:6][cH:7][cH:8][c:9]2[cH:10][n:11][cH:12][cH:13][c:14]12"
mol = Chem.MolFromSmiles(smiles)
display(mol)
graph = converter.mol_to_graph(mol)

# Display some graph details
print("Nodes and their attributes:")
print(graph.nodes(data=True))
print("\nEdges and their attributes:")
print(graph.edges(data=True))

In [ ]:
from syntemp.SynVis.chemical_graph_visualizer import ChemicalGraphVisualizer

vis = ChemicalGraphVisualizer(seed=42)
vis.graph_vis(graph)

In [ ]:
node_attributes = {
    "element": "element",
    "charge": "charge",
    "atom_atom_map": "atom_atom_map",
}
edge_attributes = {"order": "order"}
converter = GraphToMol(node_attributes, edge_attributes)

# Convert graph to RDKit Mol
mol = converter.graph_to_mol(graph)
display(mol)
print(Chem.MolToSmiles(mol))

# **6. Graph Rules alignment**

In [ ]:
import sys

sys.path.append("../")
from syntemp.SynUtils.utils import load_database, load_from_pickle
from syntemp.SynRule.rules_extraction import RuleExtraction
from syntemp.SynVis.chemical_graph_visualizer import ChemicalGraphVisualizer
from syntemp.SynRule.rule_cluster import RuleCluster

process_graph_data[0]

In [ ]:
node_label_names = ["element", "aromatic", "charge"]
node_label_names = ["element", "charge"]
naive_cluster = RuleCluster(
    node_label_names=node_label_names,
    node_label_default=["*", False, 0],
    edge_attribute="order",
)

its_graph_rules_cluster = naive_cluster.process_rules_clustering(
    process_graph_data, rule_column="GraphRules"
)
naive = [
    {"R-id": d["R-id"], "naive_cluster": d["naive_cluster"]}
    for d in its_graph_rules_cluster
]
r_id = [d["R-id"] for d in naive]
its_graph_rules_cluster[0]

In [ ]:
import pandas as pd

pd.DataFrame(naive)["naive_cluster"].value_counts()

In [ ]:
from SynTemp.SynUtils.utils import stratified_random_sample

sampled_data = stratified_random_sample(
    its_graph_rules_cluster, property_key="naive_cluster", samples_per_class=1, seed=42
)

In [ ]:
vis.vis_three_graph(sampled_data[13]["GraphRules"])

# **7. Unbalance reaction**

In [ ]:
import sys

sys.path.append("../../")
from SynTemp.SynUtils.utils import load_database
import pandas as pd

unb = load_database("../../Data/AAM/natcomm/natcomm_aam_reactions.json.gz")

mapper_name = ["graphormer", "local_mapper", "rxn_mapper"]

from SynTemp.SynITS.its_extraction import ITSExtraction

correct, incorrect = ITSExtraction.parallel_process_smiles(
    unb, mapper_name, n_jobs=4, threshold=2
)

len(correct), len(incorrect)

# **8. Arbitrary Hydrogen**

In [ ]:
from rdkit import Chem

test = "[CH:10]=1[CH:11]=[CH:12][C:7](=[CH:8][CH:9]=1)[N:5]([OH:6])[C:3](=[O:4])[O:2][CH3:1].[Cl:16][C:14]([Cl:13])([Cl:15])[C:17]#[N:18]>>[Cl:13][C:14]([Cl:16])([Cl:15])[C:17]([NH:18][C:12]=1[C:7](=[CH:8][CH:9]=[CH:10][CH:11]=1)[NH:5][C:3]([O:2][CH3:1])=[O:4])=[O:6]"
from SynTemp.SynVis.chemical_reaction_visualizer import ChemicalReactionVisualizer

vis = ChemicalReactionVisualizer()
vis.visualize_reaction(test, show_atom_map=True, img_size=(1000, 300))

In [ ]:
from SynTemp.SynVis.its_visualizer import ITSVisualizer
from IPython.display import Image

its_vis = ITSVisualizer(test)
img_sample = Image(its_vis.draw_product_with_modified_bonds(showAtomMaps=True))
img_sample

In [ ]:
test_arbitrary = [{"R-id": "C1", "mapper": test}]
from SynTemp.SynITS.its_extraction import ITSExtraction

mapper_names = ["mapper"]
correct, incorrect = ITSExtraction.parallel_process_smiles(
    test_arbitrary, mapper_names=mapper_names, check_method="RC", threshold=0
)
react_graph, product_graph, rule_graph = (
    correct[0]["ITSGraph"][0],
    correct[0]["ITSGraph"][1],
    correct[0]["ITSGraph"][2],
)

In [ ]:
from SynTemp.SynITS.its_hadjuster import ITSHAdjuster
from SynTemp.SynITS.its_construction import ITSConstruction

variations = ITSHAdjuster.add_hydrogen_nodes_multiple(react_graph, product_graph)
its_list = [ITSConstruction.ITSGraph(i[0], i[1]) for i in variations]

group_1, group_2 = variations[0] + (its_list[0],), variations[1] + (its_list[1],)
rules_1 = RuleExtraction.extract_reaction_rules(*group_1, extend=False, n_knn=1)
rules_2 = RuleExtraction.extract_reaction_rules(*group_2, extend=False, n_knn=1)

In [ ]:
from SynTemp.SynVis.chemical_graph_visualizer import ChemicalGraphVisualizer

vis_graph = ChemicalGraphVisualizer(seed=42)
vis_graph.vis_three_graph(
    rules_1, left_graph_title="L", right_graph_title="R", k_graph_title="K"
)

In [ ]:
vis_graph.vis_three_graph(
    rules_2, left_graph_title="L", right_graph_title="R", k_graph_title="K"
)